# Detección de Objetos: YOLO, 

In [2]:
# Pesos descargados desde https://pjreddie.com/darknet/yolo/
!wget www.ivan-sipiran.com/downloads/yolo.zip
!unzip yolo.zip

--2020-11-29 13:16:22--  http://www.ivan-sipiran.com/downloads/yolo.zip
Resolving www.ivan-sipiran.com (www.ivan-sipiran.com)... 66.96.149.31
Connecting to www.ivan-sipiran.com (www.ivan-sipiran.com)|66.96.149.31|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233686170 (223M) [application/zip]
Saving to: ‘yolo.zip.1’

yolo.zip.1          100%[===================>] 222.86M   107MB/s    in 2.1s    

2020-11-29 13:16:24 (107 MB/s) - ‘yolo.zip.1’ saved [233686170/233686170]

Archive:  yolo.zip
replace config/coco.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: config/coco.data        
  inflating: config/coco.names       
  inflating: config/download_weights.sh  
  inflating: config/yolov3-tiny.cfg  
  inflating: config/yolov3.cfg       
  inflating: config/yolov3.weights   
  inflating: images/blueangels-det.jpg  
  inflating: images/blueangels.jpg   
  inflating: images/Intersection-Counts.jpg  
  inflating: images/olympic-trials.jpg  
  inflating: ut

In [7]:
from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

## 1. Cargando el modelo y los pesos

In [8]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
nms_thres=0.4

model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path) # 80 clases
Tensor = torch.cuda.FloatTensor

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
classes

In [ ]:
len(classes)

In [13]:
def detect_image(img):
  ''' Detecta objetos en una imagen '''
  # escalar y rellenar imagen
  ratio = min(img_size/img.size[0], img_size/img.size[1])
  imw = round(img.size[0] * ratio)
  imh = round(img.size[1] * ratio)
  img_transforms = transforms.Compose([transforms.Resize((imh, imw)),
        transforms.Pad((max(int((imh-imw)/2),0),
                        max(int((imw-imh)/2),0),
                        max(int((imh-imw)/2),0),
                        max(int((imw-imh)/2),0)),
                      (128,128,128)),
        transforms.ToTensor(),
        ])
  # convertir imagen a Tensor
  image_tensor = img_transforms(img).float()
  image_tensor = image_tensor.unsqueeze_(0)
  input_img = Variable(image_tensor.type(Tensor))
  # ejecutar inferencia en el modelo y obtener detecciones
  with torch.no_grad():
      detections = model(input_img)
      detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
  return detections[0]

## 2. Proyecto

In [40]:
import cv2
import imutils
import timeit

In [99]:
vs = cv2.VideoCapture('proy40.mp4')

In [100]:
conf_thres = 0.1
nms_thres = 0.1

writer = None
start = time.time()
i=1
k=0
while i < 10000:
  (grab, frame) = vs.read()

  
  i= i+1
  
  if not grab:
    break

  prev_time = time.time()
  
  newImg = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

  img = Image.fromarray(newImg)
  detections = detect_image(img)
  inference_time = datetime.timedelta(seconds=time.time() - prev_time)
  print ('Inference Time: %s' % (inference_time))

  # generar colores para los bounding-box
  cmap = plt.get_cmap('tab20b')
  colors = [cmap(i) for i in np.linspace(0, 1, 20)]

  img = np.array(img)
  
  pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
  pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
  unpad_h = img_size - pad_y
  unpad_w = img_size - pad_x

  if detections is not None:
      unique_labels = detections[:, -1].cpu().unique()
      n_cls_preds = len(unique_labels)
      bbox_colors = random.sample(colors, n_cls_preds)
      
     
      # recorrer detecciones y dibujar bounding boxes
      for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
          box_h = ((y2 - y1) / unpad_h) * img.shape[0]
          box_w = ((x2 - x1) / unpad_w) * img.shape[1]
          y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
          x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]
          color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]

          newImg = cv2.rectangle(newImg, (x1,y1),(x1+box_w, y1+box_h), color, 2)
          newImg = cv2.putText(newImg, classes[int(cls_pred)], (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
      if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        writer = cv2.VideoWriter("output.avi", fourcc, 30, (newImg.shape[1],newImg.shape[0]), True)
        
      writer.write(cv2.cvtColor(newImg, cv2.COLOR_RGB2BGR))

writer.release()



Inference Time: 0:00:00.049900
Inference Time: 0:00:00.049653
Inference Time: 0:00:00.049578
Inference Time: 0:00:00.049622
Inference Time: 0:00:00.038618
Inference Time: 0:00:00.036236
Inference Time: 0:00:00.036487
Inference Time: 0:00:00.034910
Inference Time: 0:00:00.035229
Inference Time: 0:00:00.033859
Inference Time: 0:00:00.033270
Inference Time: 0:00:00.030293
Inference Time: 0:00:00.030411
Inference Time: 0:00:00.030521
Inference Time: 0:00:00.030564
Inference Time: 0:00:00.031823
Inference Time: 0:00:00.029539
Inference Time: 0:00:00.030318
Inference Time: 0:00:00.031432
Inference Time: 0:00:00.030250
Inference Time: 0:00:00.031200
Inference Time: 0:00:00.030841
Inference Time: 0:00:00.029318
Inference Time: 0:00:00.030306
Inference Time: 0:00:00.032107
Inference Time: 0:00:00.031975
Inference Time: 0:00:00.031768
Inference Time: 0:00:00.029172
Inference Time: 0:00:00.029504
Inference Time: 0:00:00.029936
Inference Time: 0:00:00.029693
Inference Time: 0:00:00.030075
Inferenc

AttributeError: ignored

Se toma el valor de i (para contabilizar objetos) y se multiplica por un factor dependiendo de la velocidad. Factor según velocidad:


1.   40Km ---> Factor =  1
2.   60Km ---> Factor =  1.5

1.   80Km ---> Factor =  2 
2.   80Km ---> Factor =  2.5

Dada la repititividad se resta un factor de 1461

Cantidad de objetos : i*factor - 1461



In [102]:
# Cantidad de frames
print(i*1-1461)

23
